# Imports

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

# Environment Data

In [2]:
#Imported data taken from the robot's readings of the entire environment.

data = [[103, 107, 130, 1638, 1200, 1181, 1183, 1210, 2550, 2138, 2151, 2193, 2550, 2550, 147, 121, 121, 143, 205, 130, 109], [103, 109, 128, 1654, 1219, 1189, 1183, 1202, 2550, 2435, 2151, 2550, 2550, 2550, 2550, 122, 122, 135, 156, 197, 112], [103, 112, 149, 1649, 1219, 1201, 1201, 1234, 2439, 2147, 2212, 2154, 2550, 2550, 122, 107, 103, 126, 191, 135, 112], [103, 109, 149, 1641, 1215, 1201, 1202, 1229, 2457, 2161, 2550, 2550, 2550, 2550, 121, 103, 103, 121, 145, 130, 109], [107, 118, 158, 1649, 1201, 1196, 1202, 1246, 2447, 2147, 2550, 2550, 2550, 2550, 118, 103, 109, 135, 195, 126, 112], [95, 94, 113, 1565, 877, 857, 855, 865, 1080, 1089, 103, 99, 113, 147, 474, 447, 443, 454, 490, 517, 103], [95, 101, 126, 1566, 871, 855, 857, 877, 939, 121, 103, 103, 121, 139, 464, 446, 446, 460, 496, 517, 101], [86, 93, 112, 1563, 871, 857, 861, 871, 2550, 137, 114, 112, 126, 154, 464, 447, 443, 455, 483, 121, 95], [84, 86, 113, 1563, 873, 857, 857, 872, 1098, 137, 114, 114, 130, 2550, 469, 447, 443, 455, 479, 534, 93], [84, 86, 114, 1557, 871, 857, 857, 873, 1196, 135, 113, 113, 135, 156, 464, 443, 443, 455, 496, 112, 86], [122, 121, 154, 599, 536, 515, 515, 532, 586, 1099, 1099, 1809, 2176, 909, 804, 788, 784, 796, 825, 1179, 130], [113, 122, 145, 582, 538, 519, 515, 534, 582, 1100, 1107, 1799, 2167, 909, 804, 780, 780, 794, 819, 804, 128], [112, 113, 138, 582, 542, 517, 517, 538, 578, 1105, 1115, 2155, 1337, 909, 800, 784, 778, 792, 878, 864, 118], [120, 120, 156, 580, 540, 518, 525, 542, 590, 1091, 1106, 2133, 1317, 835, 794, 778, 778, 788, 810, 164, 122], [118, 120, 147, 603, 551, 518, 525, 537, 597, 1113, 1107, 2138, 1330, 855, 796, 780, 773, 788, 810, 2550, 122], [455, 460, 479, 491, 536, 511, 511, 527, 572, 586, 856, 1513, 1808, 2550, 915, 109, 103, 122, 1173, 1173, 464], [460, 460, 491, 696, 532, 517, 517, 536, 578, 578, 856, 1787, 1528, 2550, 915, 107, 107, 122, 1196, 511, 464], [454, 455, 492, 726, 532, 517, 517, 534, 595, 595, 849, 1787, 1529, 2550, 915, 107, 107, 120, 1179, 1179, 460], [454, 456, 483, 706, 527, 517, 519, 540, 580, 580, 861, 1503, 1837, 2550, 1301, 103, 101, 120, 1179, 1179, 455], [454, 460, 483, 500, 544, 525, 525, 537, 597, 609, 853, 1490, 1830, 2550, 915, 101, 94, 114, 1166, 1173, 462], [95, 94, 118, 162, 122, 101, 101, 113, 2550, 1866, 1549, 1805, 2550, 2550, 483, 447, 443, 455, 500, 500, 99], [86, 93, 114, 164, 113, 99, 103, 122, 2550, 1848, 1823, 1809, 2550, 2550, 479, 447, 448, 464, 693, 2550, 95], [82, 86, 112, 164, 114, 94, 95, 118, 2550, 1611, 1835, 1821, 1808, 2550, 483, 454, 452, 460, 509, 2550, 91], [86, 93, 121, 156, 107, 93, 99, 128, 2550, 1650, 1825, 1805, 2550, 2550, 473, 454, 454, 469, 509, 114, 86], [94, 101, 126, 166, 112, 93, 94, 114, 2550, 1626, 1823, 1801, 2550, 2550, 483, 454, 454, 464, 706, 2550, 101], [428, 435, 464, 488, 454, 436, 439, 457, 506, 1866, 1829, 1876, 1885, 2550, 131, 107, 107, 120, 149, 454, 437], [427, 433, 469, 510, 454, 439, 439, 460, 500, 1856, 1825, 1841, 1876, 2550, 2550, 107, 103, 126, 490, 454, 433], [435, 439, 460, 518, 452, 437, 439, 460, 504, 1864, 1831, 1827, 1885, 2550, 2550, 109, 103, 122, 162, 462, 441], [439, 445, 483, 469, 445, 437, 437, 454, 502, 1830, 1817, 1843, 1862, 2550, 135, 112, 112, 126, 499, 464, 446], [439, 446, 475, 481, 447, 439, 439, 456, 527, 1852, 1823, 1831, 1864, 2550, 2550, 107, 107, 122, 848, 464, 445]]

Zone_1 = [data[0], data[1], data[2], data[3], data[4]]
Zone_2 = [data[5], data[6], data[7], data[8], data[9]]
Zone_3 = [data[10], data[11], data[12], data[13], data[14]]
Zone_4 = [data[15], data[16], data[17], data[18], data[19]]
Zone_5 = [data[20], data[21], data[22], data[23], data[24]]
Zone_6 = [data[25], data[26], data[27], data[28], data[29]]

# Training Environment

In [3]:
class TrainZoneEnv(Env):
    def __init__(self):
        # Actions we can take: Guess 1 of 6 zones
        self.action_space = Discrete(6)
        #  array
        self.observation_space = Box(low=np.array([0]), high=np.array([5]))
        # Set initial Zone Guess
        self.state = 0
        # Import the real Zone's sensor data
        Zone = random.randint(0,5)
        Num = 5*(Zone)
        sample = data[Num]
        self.sensor_data = sample
        # Set amount of guesses per run
        self.guess_length = 10
        
    def step(self, action):
        # Difine the sensor values for each Zone
        sensor_zone = [Zone_1, Zone_2, Zone_3, Zone_4, Zone_5, Zone_6]
        #apply the action
        self.state = action
        # Reduce guess length by 1
        self.guess_length -= 1 
        
        # Calculate reward
        # if the sensor data is in the chosen zone class, reward = 1
        if self.sensor_data in sensor_zone[self.state]: 
            reward = 1
            correct = True
        else: 
            reward = -1
            correct = False
        
        # Check if guessing is done
        if self.guess_length <= 0 or correct == True: 
            done = True
        else:
            done = False
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset Initial Zone Guess
        self.state = 0
        # Import a new Zone's sensor data
        Zone = random.randint(0,5)
        Num = 5*(Zone)
        sample = data[Num]
        self.sensor_data = sample
        # Reset Guess amount
        self.guess_length = 10
        return self.state

In [4]:
env = TrainZoneEnv()

episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} Zone guess:{}'.format(episode, score, n_state+1))

Episode:1 Score:-1 Zone guess:4
Episode:2 Score:-4 Zone guess:4
Episode:3 Score:0 Zone guess:1
Episode:4 Score:1 Zone guess:6
Episode:5 Score:-3 Zone guess:6
Episode:6 Score:0 Zone guess:3
Episode:7 Score:0 Zone guess:2
Episode:8 Score:-1 Zone guess:5
Episode:9 Score:-10 Zone guess:2
Episode:10 Score:1 Zone guess:3


C:\Users\Dylan\anaconda3\lib\site-packages\gym\spaces\box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


# Model Building

In [5]:
states = env.observation_space.shape
actions = env.action_space.n

def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [6]:
del model

NameError: name 'model' is not defined

In [7]:
model = build_model(states, actions)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 6)                 150       
                                                                 
Total params: 798
Trainable params: 798
Non-trainable params: 0
_________________________________________________________________


In [8]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

# Model Training

In [9]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

C:\Users\Dylan\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 7:38 - reward: -1.0000

C:\Users\Dylan\anaconda3\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   40/10000 [..............................] - ETA: 2:11 - reward: -0.7500

C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 16 + 1) 

10000/10000 [==============================] - 55s 5ms/step - reward: -0.6666
1991 episodes - episode_reward: -3.347 [-10.000, 1.000] - loss: 1.202 - mae: 2.216 - mean_q: -2.180

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 58s 6ms/step - reward: -0.6578
2008 episodes - episode_reward: -3.274 [-10.000, 1.000] - loss: 1.311 - mae: 2.320 - mean_q: -2.332

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 60s 6ms/step - reward: -0.6392
2098 episodes - episode_reward: -3.047 [-10.000, 1.000] - loss: 1.224 - mae: 2.206 - mean_q: -2.175

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 62s 6ms/step - reward: -0.6544
2030 episodes - episode_reward: -3.223 [-10.000, 1.000] - loss: 1.255 - mae: 2.234 - mean_q: -2.211

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 63s 6ms/step - reward: -0.6360
done, took 297.945 seconds


# Model Testing

In [10]:
class TestZoneEnv(Env):
    def __init__(self):
        # Actions we can take: Guess 1 of 6 zones
        self.action_space = Discrete(6)
        #  array
        self.observation_space = Box(low=np.array([0]), high=np.array([5]))
        # Set initial Zone Guess
        self.state = 0
        # Import the real Zone's sensor data
        sample = data[Num]
        self.sensor_data = sample
        # Set amount of guesses per run
        self.guess_length = 10
        
    def step(self, action):
        # Difine the sensor values for each Zone
        sensor_zone = [Zone_1, Zone_2, Zone_3, Zone_4, Zone_5, Zone_6]
        #apply the action
        self.state = action
        # Reduce guess length by 1
        self.guess_length -= 1 
        
        # Calculate reward
        # if the sensor data is in the chosen zone class, reward = 1
        if self.sensor_data in sensor_zone[self.state]: 
            reward = 1
            correct = True
        else: 
            reward = -1
            correct = False
        
        # Check if guessing is done
        if self.guess_length <= 0 or correct == True: 
            done = True
        else:
            done = False
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset Initial Zone Guess
        self.state = 0
        # Import a new Zone's sensor data
        sample = data[Num]
        self.sensor_data = sample
        # Reset Guess amount
        self.guess_length = 10
        return self.state

In [49]:
# Choose the zone the Agent is in.

Zone = 1
Num = 5*(Zone -1) + 3

In [50]:
# Test to determine if the Agent knows where it is.
env = TestZoneEnv()

scores = dqn.test(env, nb_episodes=2, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 2 episodes ...
Episode 1: reward: 0.000, steps: 2
Episode 2: reward: 0.000, steps: 2
0.0
